In [ ]:
# 📦 Установка (если не предустановлено в Kaggle, но обычно есть)
# pip install scikit-image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import io, color, filters, feature, exposure, util, transform
from skimage.measure import label, regionprops
from skimage.feature import hog
from skimage.color import rgb2gray

# 📥 Загрузка изображения
img = io.imread('your_image_path.jpg')  # RGB изображение
gray = rgb2gray(img)                    # grayscale

# 📊 Визуализация
plt.imshow(gray, cmap='gray')
plt.title('Grayscale')
plt.axis('off')
plt.show()

# 📌 1. Применение фильтра (edge detection)
edges = filters.sobel(gray)
plt.imshow(edges, cmap='gray')
plt.title('Sobel edges')
plt.axis('off')
plt.show()

# 📌 2. HOG фичи (для классификации / PCA)
hog_feats, hog_img = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                         visualize=True, feature_vector=True)
plt.imshow(hog_img, cmap='gray')
plt.title('HOG visualization')
plt.axis('off')
plt.show()

# 📌 3. Threshold + бинаризация
from skimage.filters import threshold_otsu
thresh = threshold_otsu(gray)
binary = gray > thresh
plt.imshow(binary, cmap='gray')
plt.title('Binarized image (Otsu)')
plt.axis('off')
plt.show()

# 📌 4. Resize
resized = transform.resize(gray, (64, 64))
print(resized.shape)

# 📌 5. Feature extraction (mean, std, etc.)
mean = np.mean(gray)
std = np.std(gray)
skewness = pd.Series(gray.flatten()).skew()
print("Mean:", mean, "| Std:", std, "| Skew:", skewness)

# 📌 6. Для классификации:
# Сделать feature vector из hog, mean, std и использовать в sklearn

features = np.concatenate([[mean, std, skewness], hog_feats])


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Пример: X — это твой массив признаков (numpy array или pandas DataFrame)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # PCA любит стандартизованные данные

pca = PCA(n_components=50)  # можно указать число компонент
X_pca = pca.fit_transform(X_scaled)

print("Было:", X.shape)
print("Стало:", X_pca.shape)


In [ ]:
from skimage.io import imread_collection
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage.transform import resize

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

import numpy as np
import os

# 📁 Загрузи изображения (пример: 2 класса по папкам 'class0' и 'class1')
def load_images_from_folder(folder, label, size=(64, 64)):
    X, y = [], []
    for filename in os.listdir(folder):
        img = imread_collection(os.path.join(folder, filename))[0]
        img = rgb2gray(resize(img, size))
        features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        X.append(features)
        y.append(label)
    return X, y

# 🎯 Загрузка данных
X0, y0 = load_images_from_folder('data/class0', 0)
X1, y1 = load_images_from_folder('data/class1', 1)

X = np.array(X0 + X1)
y = np.array(y0 + y1)

# 🔁 Train/Val split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# 🧠 Классификация
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# 📊 Оценка
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))
print("Macro F1:", f1_score(y_val, y_pred, average='macro'))
